In [1]:
!pip install -U -qq transformers accelerate exllamav2 langchain
!apt-get update && apt-get install git-lfs
!git lfs install

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [634 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]        
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [28.5 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]     
Get:9 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:11 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiv

In [2]:
!wget https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf

--2023-12-14 05:23:09--  https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.244.202.73, 18.244.202.68, 18.244.202.60, ...
Connecting to huggingface.co (huggingface.co)|18.244.202.73|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/ac/ba/acba0635d39a127379c2c6ae1cefacc586bf413e8b044c5ca82daade27d7d503/9193684683657e90707087bd1ed19fd0b277ab66358d19edeadc26d6fdec4f53?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf%3B+filename%3D%22mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf%22%3B&Expires=1702790589&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjc5MDU4OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2FjL2JhL2FjYmEwNjM1ZDM5YTEyNzM3OWMyYzZhZTFjZWZhY2M1ODZiZjQxM2U4YjA0NGM1Y2E4MmRhYWRlMjd

In [3]:
#Base ctransformers with no GPU acceleration
!pip install llama-cpp-python
#With NVidia CUDA acceleration
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
#Or with OpenBLAS acceleration
!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python
# Or with CLBLast acceleration
!CMAKE_ARGS="-DLLAMA_CLBLAST=on" pip install llama-cpp-python
#Or with AMD ROCm GPU acceleration (Linux only)
!CMAKE_ARGS="-DLLAMA_HIPBLAS=on" pip install llama-cpp-python
#Or with Metal GPU acceleration for macOS systems only
!CMAKE_ARGS="-DLLAMA_METAL=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 37.9 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 11.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.23-cp310-cp310-manylinux_2_35_x86_64.whl size=2075036 sha256=14282d7b6e4d00f5dc7e3aef7b1d2f4eccacac33b9e08347173ef353f37607a4
  Stored in directory: /root/.cache/pip/wheels/8b/7c/23/5851b51f3b9088d6f7a5ba6d27b3494aa4940bf291b43ee04d
Successfully built llama-cpp-python


In [4]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

chat_model = LlamaCpp(
    model_path="/workspace/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from /workspace/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:          blk.0.ffn_gate.0.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    2:          blk.0.ffn_down.0.weight q4_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_up.0.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:          blk.0.ffn_gate.1.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:          blk.0.ffn_down.1.weight q4_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.ffn_up.1.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:          blk.0.ffn_gate.2.weight q4_K     [  4096,

In [5]:
import requests

def get_weather_data(coordinates):
    """
    Fetches weather data from the Open-Meteo API for the specified latitude and longitude.

    Args:
    coordinates (tuple): Latitude and longitude of the location.

    Returns:
    float: Current temperature at the queried location.
    """
    latitude, longitude = coordinates
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current": "temperature_2m,wind_speed_10m",
        "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m",
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return f"""Temperature(C): {response.json()["current"]["temperature_2m"]}"""
    else:
        return {
            "error": "Failed to fetch data, status code: {}".format(
                response.status_code
            )
        }


def get_coordinates_from_city(city_name):
    """
    Retrieves latitude and longitude for the specified city name using Maps.co Geocoding API.

    Args:
    city_name (str): Name of the city.

    Returns:
    tuple: Latitude and longitude of the city.
    """
    base_url = "https://geocode.maps.co/search"
    params = {"q": city_name}

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data:
            # Assuming the first result is the most relevant
            return data[0]["lat"], data[0]["lon"]
        else:
            return {"error": "No data found for the specified city name."}
    else:
        return {
            "error": "Failed to fetch data, status code: {}".format(
                response.status_code
            )
        }


In [6]:
prompt_template = \
'''
Function:
def get_weather_data(coordinates):
    """
    Fetches weather data from the Open-Meteo API for the given latitude and longitude.

    Args:
    coordinates (tuple): The latitude of the location.

    Returns:
    float: The current temperature in the coordinates you've asked for
    """

Function:
def get_coordinates_from_city(city_name):
    """
    Fetches the latitude and longitude of a given city name using the Maps.co Geocoding API.

    Args:
    city_name (str): The name of the city.

    Returns:
    tuple: The latitude and longitude of the city.
    """

User Query: {query}<human_end>

'''


In [9]:
from langchain.schema.messages import HumanMessage, SystemMessage

query = "Whats's the weather like in Seattle right now?"
messages = [
    HumanMessage(content=prompt_template.format(query=query))
]

result = llm.invoke(messages)

print(result)

Llama.generate: prefix-match hit


To answer this question, we need to use both functions that have been provided. 

Firstly, we need to find out the coordinates of the city "Seattle". To achieve this, we can call and execute the function `get_coordinates_from_city("Seattle")")` . This will return the coordinates of Seattle in the format of a tuple e.g. (47.6062, -122.3321)).

Secondly, now that we have the coordinates of Seattle, we can now use these coordinates to call and execute the function `get_weather_data((47.6062, -122.3321)))` . This will return the current weather data for Seattle in the format of a dictionary e.g. {"temperature": 50.5, "pressure": 1013.0, "humidity": 78.0, ...}

Finally, now that we have the current weather data for Seattle in the format of a dictionary e.g. {"temperature": 50.5, "pressure": 1013.0, "humidityTo answer this question, we need to use both functions that have been provided. 

Firstly, we need to find out the coordinates of the city "Seattle". To achieve this, we can call and exe


llama_print_timings:        load time =  163997.53 ms
llama_print_timings:      sample time =     198.22 ms /   256 runs   (    0.77 ms per token,  1291.53 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  945140.75 ms /   256 runs   ( 3691.96 ms per token,     0.27 tokens per second)
llama_print_timings:       total time =  948049.16 ms


# CHAIN 

In [14]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
)
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


def call_function(text: str):
    return eval(text)


prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template(prompt_template),
    ]
)

chain = (
    {"query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
result = chain.invoke("What are the latitude and longitude of Osaka?")
print(result)

Llama.generate: prefix-match hit


To answer this query, you can use the `get_coordinates_from_city()` function defined above. Here's an example implementation:

```python
def get_osaka_coordinates():
    # Replace 'Osaka' with the city name you want to fetch coordinates for.
    city_name = 'Osaka'

    # Fetch coordinates using the `get_coordinates_from_city()` function defined earlier.
    coordinates = get_coordinates_from_city(city_name)

    return coordinates

# Call the function to fetch and print the latitude and longitude of Osaka.
osaka_coordinates = get_osaka_coordinates()
print('Osaka Coordinates:', osaka_coordinates)
```

When you run this code, it should print out the latitude and longitude of Osaka.To answer this query, you can use the `get_coordinates_from_city()` function defined above. Here's an example implementation:

```python
def get_osaka_coordinates():
    # Replace 'Osaka' with the city name you want to fetch coordinates for.
    city_name = 'Osaka'

    # Fetch coordinates using the `get_coord


llama_print_timings:        load time =  163997.53 ms
llama_print_timings:      sample time =     146.96 ms /   198 runs   (    0.74 ms per token,  1347.34 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  731916.25 ms /   198 runs   ( 3696.55 ms per token,     0.27 tokens per second)
llama_print_timings:       total time =  733972.14 ms
